I'm going to use `gumpy` in a few places, but that is because I am used to it; we can easily use more standard tools for most functions

In [18]:
import gumpy, copy, numpy, yaml, json, subprocess, os

from tqdm import tqdm

In [2]:
covid_reference=gumpy.Genome('MN908947.3.gbk')
covid_reference

MN908947
MN908947.3
Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
29903 bases
attaaa...aaaaaa
metadata for all genes/loci have been included

In [4]:
covid_reference.save_fasta('MN908947.fasta',overwrite_existing=True,description='MN908947')

## Using `samtools` and `pyfastaq` to produce a perfect FASTQ file

In [6]:
with open("covid-artic-v3.json") as f:                                                                                 
    json_data = json.load(f)        

Run it once to generate an index

In [9]:
ref_fasta = "MN908947.fasta"
tmp_ref = "MN908947.tmp"
reads_out = "MN908947.fastq"

subprocess.check_output(f"samtools faidx {ref_fasta}", shell=True)

if os.path.isfile(reads_out):
    os.unlink(reads_out) 
    
for name,d in tqdm(json_data['amplicons'].items()):

    # coords are 0-based
    start, end = d["start"] + 1, d["end"] + 1                                                                                               
    
    subprocess.check_output(f"samtools faidx {ref_fasta} MN908947:{start}-{end} > {tmp_ref}", shell=True) 
    
    # the arguments below are 
    #  - Mean insert size of read pairs
    #  - Standard devation of insert size
    #  - Mean coverage of the reads
    #  - Length of each read
    
    subprocess.check_output(f"fastaq to_perfect_reads {tmp_ref} - 300 10 1000 175 >> {reads_out}", shell=True)

os.unlink(tmp_ref) 
   

100%|███████████████████████████████████████████| 98/98 [00:40<00:00,  2.44it/s]


In [10]:
fastq1 = 'MN908947_1.fastq'
fastq2 = 'MN908947_2.fastq'
subprocess.check_output(f"fastaq deinterleave {reads_out} {fastq1} {fastq2}", shell=True)

b''

## Parsing `phe-genomics/variant_definitions` which is used by `aln2type`

This is the definition file for Delta

In [12]:
with open('empathy-serve.yml') as f:
    variant_definitions=yaml.safe_load(f)

Delta is a bit easier as this definition only contains SNPs

In [13]:
for i in variant_definitions['variants']:
    assert i['type']=='SNP' 

Because there are no INDELS we could simply use a `str` but I'm going to use `gumpy` (since it would also cope with indels and is more explicit)

In [14]:
delta_variant=copy.deepcopy(covid_reference)

for i in tqdm(variant_definitions['variants']):
    
    # build the mask to isolate the nucleotide
    mask=delta_variant.nucleotide_index==i['one-based-reference-position']
    
    # check the definition agrees with the reference sequence
    assert delta_variant.nucleotide_sequence[mask]==i['reference-base'].lower()
    
    # only now mutate the base
    delta_variant.nucleotide_sequence[mask]=i['variant-base']
    
# check if they are different
delta_variant==covid_reference    

100%|███████████████████████████████████████████| 8/8 [00:00<00:00, 5537.03it/s]


False

In [15]:
delta_variant.save_fasta('Delta.fasta',\
                         fixed_length=False,\
                         overwrite_existing=True,\
                         description='Delta')

In [16]:
ref_fasta = "Delta.fasta"
tmp_ref = "Delta.ref"
reads_out = "Delta.fastq"

subprocess.check_output(f"samtools faidx {ref_fasta}", shell=True)

if os.path.isfile(reads_out):
    os.unlink(reads_out) 

for name,d in tqdm(json_data['amplicons'].items()):

    # coords are 0-based
    start, end = d["start"] + 1, d["end"] + 1                                                                                               
    
    subprocess.check_output(f"samtools faidx {ref_fasta} Delta:{start}-{end} > {tmp_ref}", shell=True) 
    
    # the arguments below are 
    #  - Mean insert size of read pairs
    #  - Standard devation of insert size
    #  - Mean coverage of the reads
    #  - Length of each read
    
    subprocess.check_output(f"fastaq to_perfect_reads {tmp_ref} - 300 10 1000 175 >> {reads_out}", shell=True)

os.unlink(tmp_ref)  
    

100%|███████████████████████████████████████████| 98/98 [00:40<00:00,  2.40it/s]


In [17]:
fastq1 = 'Delta_1.fastq'
fastq2 = 'Delta_2.fastq'
subprocess.check_output(f"fastaq deinterleave {reads_out} {fastq1} {fastq2}", shell=True)

b''